In [8]:
%pylab inline
import pandas as pd
import os 

Populating the interactive namespace from numpy and matplotlib


Construcción base de datos:

In [9]:
#Inicializo las columnas con los nombres de las localidades que ya tienen datos
localidades=os.listdir("Votaciones_localidades")
Nombreslocalidades=[]
for i in localidades:
    Nombreslocalidades.append(i[:-7])
df1 = pd.DataFrame(columns=[Nombreslocalidades])
df1= df1.rename_axis(None)
print (localidades)

['ANTONIO_NARINO_OK.csv', 'CANDELARIA_OK.csv', 'CARCELES_OK.csv', 'CHAPINERO_OK.csv', 'CORFERIAS_OK.csv', 'MARTIRES_OK.csv', 'RAFAEL_URIBE_URIBE_OK.csv', 'SANTA_FE_OK.csv', 'SUMAPAZ_OK.csv', 'TUNJUELITO_OK.csv']


In [10]:
#Algoritmo que ingresa los datos a la tabla, la tabla solo contiene las localidades reportadas.
Index=('P_Liberal','P_Conservador','P_Opcion_Ciudadana','P_Cambio_Radical','P_Alianza_Verde','P_MIRA','P_de_la_U','Polo','P_Centro_Democratico','P_SOMOS','FARC','Colombia_Justa_Libres','SI_SE_PUEDE',"Union_con_Fortaleza","Todos_somos_Colombia",'Coalicion_Decencia','Voto_Blanco_Nacional','AICO','ASI','MAIS','SOBERANIA','MIA','RenovacionEtnica','CASIYOUREN','Voto_Blanco_Indigena','Nulos','No_Marcados',"Potencial votantes", "Referencia")
for i in localidades:
    temp=pd.read_csv("Votaciones_localidades/"+str(i))
    temp.index = Index
    df1[str(i[:-7])] = temp.iloc[:-1,1].apply(pd.to_numeric)

In [11]:
df1

,ANTONIO_NARINO,CANDELARIA,CARCELES,CHAPINERO,CORFERIAS,MARTIRES,RAFAEL_URIBE_URIBE,SANTA_FE,SUMAPAZ,TUNJUELITO
P_Liberal,3308,2194,40,3791,1978,3708,8668,3975,190,3914
P_Conservador,1735,667,47,3822,1464,1563,4020,1454,18,2499
P_Opcion_Ciudadana,554,202,23,422,554,452,1469,486,12,840
P_Cambio_Radical,3712,1791,29,5108,3428,3005,9460,2969,88,6457
P_Alianza_Verde,9563,3208,86,25557,12610,6608,17248,6713,21,10587
P_MIRA,3065,477,13,1243,1713,1472,5558,1185,5,3213
P_de_la_U,2303,822,37,2425,1780,1948,6804,1470,9,3872
Polo,3405,1814,54,4616,2905,2212,7186,3169,114,4460
P_Centro_Democratico,8614,2412,44,25213,9605,7396,15924,5034,34,9945
P_SOMOS,421,117,1,418,467,584,910,307,0,587


In [5]:
#Pendiente nuevos datos